# Coronagraph inpainting model

## Data downloading and preprocessing

In [1]:
!python3 data.py

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Importing data
LASCO-C2:22157691.fts:2003/10/28T20:10:35.655...
Importing data
LASCO-C2:22157091.fts:2003/10/20T01:55:05.979...
Importing data
LASCO-C2:22157383.fts:2003/10/24T06:54:30.881...
Importing data
LASCO-C2:22157143.fts:2003/10/20T20:54:05.924...
Importing data
LASCO-C2:22157509.fts:2003/10/26T02:30:30.992...
Importing data
LASCO-C2:22157573.fts:2003/10/27T00:06:05.441...
Importing data
LASCO-C2:22157708.fts:2003/10/29T05:24:10.719...
Importing data
LASCO-C2:22157415.fts:2003/10/24T18:06:05.473...
Importing data
LASCO-C2:22157428.fts:2003/10/24T21:54:05.535...
Importing data
LASCO-C2:22157332.fts:2003/10/23T13:33:53.413...
Importing data
LASCO-C2:22157423.fts:2003/10/24T20:56:39.970...
Importing data
LASCO-C2:22157809.fts:2003/10/30T22:09:44.810...
Importing data
LASCO-C2:22157707.fts:2003/10/29T04:15:34.061...
Importing data
LA

In [2]:
from utils import get_default_device
device = get_default_device()
print(device)

cuda


## Creating dataset

In [3]:
batch_size = 3

In [4]:
from torch.utils.data import random_split, DataLoader
from utils import DeviceDataLoader
from data import CoronagraphDataset

dataset = CoronagraphDataset('c2')
# 0.8 - 0.2
train_len = round(0.8*len(dataset))
val_len = len(dataset) - train_len

#random split
train_ds, val_ds = random_split(dataset, [train_len, val_len])

train_dl = DeviceDataLoader(DataLoader(train_ds, batch_size, True, num_workers = 4, pin_memory=True), device)
val_dl = DeviceDataLoader(DataLoader(val_ds, batch_size*2, True, num_workers = 4, pin_memory=True), device)

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
from model import CoronagraphReconstructor
from utils import to_device

model = to_device(CoronagraphReconstructor('attempt_1', in_channels = 1, deep = 2, layers = 2), device)

In [6]:
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim import Adam

model.fit(
    train_dl,
    val_dl,
    epochs = 100,
    lr = 1e-2,
    batch_size = batch_size,
    weight_decay = 1e-6,
    grad_clip = 1e-1,
    opt_func = Adam,
    lr_sched = OneCycleLR,
    saving_div = 15
)

Training - Epoch: 0:   0%|          | 0/194 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacty of 11.72 GiB of which 319.69 MiB is free. Including non-PyTorch memory, this process has 10.92 GiB memory in use. Of the allocated memory 10.71 GiB is allocated by PyTorch, and 26.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF